In [1]:
import pickle
import pandas as pd
from bs4 import BeautifulSoup
from urllib.request import Request, urlopen
import re
import time
import itertools
import string
from nltk.corpus import stopwords
import numpy as np
pd.set_option('mode.chained_assignment',None)
#pickle.dump(sg, open('sg_post_dfv2.pkl', 'bw'))

In [2]:
sg = pickle.load(open('sg_post_df.pkl', 'br'))
sg = sg[['post_number', 'short_post_txt', 'groups']]
sg = sg.drop_duplicates()
sg = sg.reset_index(drop=True)


KeyError: "['short_post_txt'] not in index"

In [3]:
print(sg.shape)
sg.sample(5)


(34410, 3)


,post_number,post_txt,groups
27544,133776,"['n ', 'io my off week of chemo and so far ou...",[' Chemotherapy']
34319,57343,['n Wondering what my life wouldu0027ve been ...,[' Obesity']
462,554161,['n Hi all! I hope someone can help.. My husb...,[' ADHD']
5466,598221,"['n I overheard my mum having iercourse, I ca...","[' Agoraphobia, Anxiety, Depression']"
10360,491892,['n Iu0027m 15 and Iu0027ve been vegetarian f...,"[' Vegan, Vegetarian']"


In [4]:
sg['post_number'] = sg['post_number'].astype(int)
sg['groups'] = sg['groups'].str.strip('[]\'').str.split(',')
sg['post_txt'] = sg['post_txt'].str.strip('[]\'')
print(sg.shape)
sg.sample(5)

(34410, 3)


,post_number,post_txt,groups
32894,57546,n My husband has early onset Parkinsons disea...,"[ Parkinsons, Caregivers]"
10725,400426,"n ', 'Seems like this board is not very activ...",[ Spina Bifida]
21140,146148,"n ', 'For anyone whou0027s feeling like they ...","[ Self Esteem, Anorexia, Bipolar]"
27874,123134,n My friend se nude photos of herself to her ...,[ Sexual Harassment]
21948,149543,"n ', 'http://www.gapsdiet.com/INTRODUCTION_DI...","[ Diverticulitis, Food Allergy, Irritable Bo..."


In [5]:
sg['post_txt'][1]

"n  I feel so trapped and helpless! My husband has borderline personality disorder and when he is well he is so lovely and caring but when he is feeling ill he emotionally abuses me, swears at me, blames me for everything, tells me Iu0027m useless and I am g...', '', "

In [6]:
sg.post_txt = sg.post_txt.apply(lambda x: x.translate(str.maketrans('','',string.punctuation)))
sg.post_txt[1]

'n  I feel so trapped and helpless My husband has borderline personality disorder and when he is well he is so lovely and caring but when he is feeling ill he emotionally abuses me swears at me blames me for everything tells me Iu0027m useless and I am g  '

In [7]:
sg = sg.reset_index(drop=True)
print(sg.shape)
sg.head(5)


(34410, 3)


,post_number,post_txt,groups
0,621322,n my 15 year old revealed to me 10 weeks ago ...,[ Friends and Family of BPD]
1,620965,n I feel so trapped and helpless My husband h...,[ Friends and Family of BPD]
2,618672,n Ive been through a pretty major episode wit...,[ Friends and Family of BPD]
3,603678,n I was told I might post in this group for s...,[ Friends and Family of BPD]
4,613714,n Hi Iu2019m new here and in the middle of di...,[ Friends and Family of BPD]


In [8]:
grp_sp = pd.DataFrame(sg.groups.values.tolist())
grp_sp.sample(10)

,0,1,2,3,4,5,6,7,8,9,...,73,74,75,76,77,78,79,80,81,82
8105,Suboxone,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
2277,Relationship,Singles,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
10340,Abuse,ADHD,Anxiety,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
14897,Herpes,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
6074,Colorectal Cancer,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
11992,Autism,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
31894,Weight Loss,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
6826,Bipolar,Jealousy,Loneliness,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
20607,Depression,Siblings,Stress,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
29918,Acne,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None


In [9]:
sg['group_1'] = grp_sp[0]
sg['group_2'] = grp_sp[1]
sg['group_3'] = grp_sp[2]
sg['group_4'] = grp_sp[3]
sg['group_5'] = grp_sp[4]
sg = sg.fillna(value = 'x')

print(len(sg))
sg.sample(5)

34410


,post_number,post_txt,groups,group_1,group_2,group_3,group_4,group_5
16755,396829,n Hey Iu0027m new here and I was diagnosed wi...,[ PCOS],PCOS,x,x,x,x
32686,57289,n I have had endometriosis for 4 years but I ...,[ Endometriosis],Endometriosis,x,x,x,x
32698,48781,n I am having a bad night I donu0027t know if...,"[ Anxiety, Relationship, Stress]",Anxiety,Relationship,Stress,x,x
19979,363083,n I am on a vegetarian 30day trial now and to...,[ Vegetarian],Vegetarian,x,x,x,x
33957,53359,n Hi everyone just waing to know your persona...,"[ Anorexia, Eating Disorder, EDNOS]",Anorexia,Eating Disorder,EDNOS,x,x


In [10]:
sg = sg.loc[sg['post_txt'].str.len() > 10]
print(len(sg))
sg.sample(5)

33775


,post_number,post_txt,groups,group_1,group_2,group_3,group_4,group_5
555,599028,n Iu0027ve been at a u0022healthyu0022 weight...,[ Anorexia],Anorexia,x,x,x,x
22501,161457,n I care for myself The more solitary the mo...,[ Loneliness],Loneliness,x,x,x,x
27384,128456,n Good morning everyone I am Jen and this is ...,[ Methadone],Methadone,x,x,x,x
23337,110190,n My husband was diagnosed in April with NHL ...,[ Non Hodgkins Lymphoma ],Non Hodgkins Lymphoma,x,x,x,x
24050,160030,n Hangin in there hoping for happiness and I...,[ HIV],HIV,x,x,x,x


In [11]:
len (sg['group_1'].unique())

248

In [12]:
stop_words = stopwords.words('english')
len(stop_words)

179

In [13]:
from nltk.tokenize import word_tokenize # Word Tokenizer
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer

def tok_n_clean (txt):
    stop_words = stopwords.words('english')
    word_tokens = word_tokenize(txt)
    lemmatizer = WordNetLemmatizer()
    lowercase_tokens = [w.lower() for w in word_tokens]
    words = [word for word in lowercase_tokens if word.isalpha()]
    words = [w for w in words if not w in stop_words]
    words = [i for i in words if len(i) > 1]
    return words

tok_n_clean(sg['post_txt'][1])



['feel',
 'trapped',
 'helpless',
 'husband',
 'borderline',
 'personality',
 'disorder',
 'well',
 'lovely',
 'caring',
 'feeling',
 'ill',
 'emotionally',
 'abuses',
 'swears',
 'blames',
 'everything',
 'tells',
 'useless']

In [14]:
lst= sg['post_txt'].tolist()
len(lst)

33775

In [15]:
def column_tok (col):
    lst= col.tolist()
    tok_lst = []
    for i in range (0, len(lst)):
        tok_lst.append(tok_n_clean(lst[i]))
    return tok_lst

sg['post_txt'] = column_tok (sg['post_txt'])
sg.sample(5)


,post_number,post_txt,groups,group_1,group_2,group_3,group_4,group_5
33972,51582,"[part, road, recovery, put, meal, plan, far, q...","[ Anorexia, Eating Disorder, EDNOS]",Anorexia,Eating Disorder,EDNOS,x,x
15200,395515,"[joined, group, need, lower, back, surgery, he...",[ Back Pain],Back Pain,x,x,x,x
33928,35865,"[heartfelt, story, readers, helped, girl, woma...","[ Hodgkins Lymphoma, Chemotherapy]",Hodgkins Lymphoma,Chemotherapy,x,x,x
34035,56118,"[picked, scabs, acne, fir, years, trying, exte...",[ Skin Picking],Skin Picking,x,x,x,x
1973,157916,"[boy, friend, found, prostate, cancer, new, help]",[ Prostate Cancer],Prostate Cancer,x,x,x,x


In [16]:
sg_X = sg[['post_txt', 'post_number']]
print(sg_X.shape)

(33775, 2)


In [17]:
g = ['group_1', 'group_2', 'group_3', 'group_4', 'group_5']
sg_y = sg[g]
print(sg_y.shape)
sg_y.sample()

(33775, 5)


,group_1,group_2,group_3,group_4,group_5
27351,Chantix,x,x,x,x


In [18]:
sg_y.dtypes

group_1    object
group_2    object
group_3    object
group_4    object
group_5    object
dtype: object

In [19]:
def clean_y (txt):
        word_tokens = word_tokenize(txt)
        lowercase_tokens = [w.lower() for w in word_tokens]
        words = [word for word in lowercase_tokens if word.isalpha()]
        words = ' '.join(words)
        return words

def clean_y_col(col):
    new = []
    c = col.tolist()
    for j in range(0, len(c)):
        new.append(clean_y(c[j]))
    return new

sg_y['group_1'] = clean_y_col(sg_y['group_1'])
sg_y['group_2'] = clean_y_col(sg_y['group_2'])
sg_y['group_3'] = clean_y_col(sg_y['group_3'])
sg_y['group_4'] = clean_y_col(sg_y['group_4'])
sg_y['group_5'] = clean_y_col(sg_y['group_5'])        
print(sg_y.shape)
sg_y.sample(5)

(33775, 5)


,group_1,group_2,group_3,group_4,group_5
17640,anemia,x,x,x,x
12186,breast cancer,x,x,x,x
9583,bedwetting,x,x,x,x
24040,hiv,x,x,x,x
18553,sexual harassment,x,x,x,x


In [22]:
okList = np.unique(sg_y[['group_1']].values)
okList

array(['abuse', 'acne', 'adderall', 'adhd', 'adoption',
       'adoption and fostering', 'agoraphobia', 'alcohol', 'alzheimers',
       'ambien', 'amputee', 'anemia', 'anger management', 'anorexia',
       'anxiety', 'arthritis', 'asperger syndrome', 'asthma', 'ativan',
       'autism', 'back pain', 'bedwetting', 'binge eating', 'bipolar',
       'birth defects', 'bisexuality', 'bladder cancer',
       'body dysmorphic disorder', 'bone cancer',
       'borderline personality disorder', 'brain cancer', 'brain injury',
       'breast cancer', 'breastfeeding', 'bulimia', 'bullying', 'burn',
       'caffeine', 'career changes', 'caregivers',
       'carpal tunnel syndrome', 'celiac disease', 'cerebral palsy',
       'cervical cancer', 'chantix', 'chemotherapy',
       'chronic fatigue syndrome', 'chronic pain', 'cirrhosis', 'cocaine',
       'codependency', 'college', 'colon cancer', 'colorectal cancer',
       'coming out', 'copd', 'crohns disease', 'cymbalta',
       'cystic fibrosis', '

In [21]:
newDF = pd.DataFrame(columns=okList)
newDF.head()

,abuse,acne,adderall,adhd,adoption,adoption and fostering,agoraphobia,alcohol,alzheimers,ambien,...,vegetarian,veterans,vicodin,video game addiction,weight loss,wellbutrin,widow,widower,xanax,zoloft


In [23]:
for x in range(sg_y.count()[0]):
    someDict={}
    for d in okList:
        rowdata=list(sg_y.iloc[x])
        if d in rowdata:
            someDict[d]=1
        else:
            someDict[d]=0
    y = newDF.append(someDict,ignore_index=True)
    
print(y.shape)
y.sample(5)

KeyboardInterrupt: 